# Save Markdown text into Vector DB

## Step-1: Config

In [1]:
import os, sys
from my_config import MY_CONFIG

## Step-2: Read Markdown

In [2]:
import os
import glob

pattern = os.path.join(MY_CONFIG.OUTPUT_FOLDER_FINAL_MD, '*.md')
md_file_count = len(glob.glob(pattern, recursive=True)) 

In [3]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir=MY_CONFIG.OUTPUT_FOLDER_FINAL_MD, recursive=False , required_exts=[".md"])
documents = reader.load_data()

print (f"Loaded {len(documents)} documents from {md_file_count} files")


/home/sujee/my-stuff/ai-alliance/data-prep-kit-examples/dpk-dev/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


Loaded 61 documents from 2 files


In [4]:
## Inspect a sample doc
print (documents[0])

Doc ID: ccf119ac-9575-4018-a812-d3212ce1c923
Text: Provided proper attribution is provided, Google hereby grants
permission to reproduce the tables and figures in this paper solely
for use in journalistic or scholarly works.


## Step-3: Create Chunks

In [5]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=MY_CONFIG.CHUNK_SIZE, chunk_overlap=MY_CONFIG.CHUNK_OVERLAP)
nodes = parser.get_nodes_from_documents(documents)
print(f"Created {len(nodes)} chunks from {len(documents)} documents")

Created 91 chunks from 61 documents


## Step-4: Setup Embedding Model

In [6]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [7]:
from llama_index.embeddings.litellm import LiteLLMEmbedding
from llama_index.core import Settings

Settings.embed_model = LiteLLMEmbedding(
        model_name=MY_CONFIG.EMBEDDING_MODEL,
        embed_batch_size=50,  # Batch size for embedding (default is 10)
    )
print (f"✅ Using embedding model: {MY_CONFIG.EMBEDDING_MODEL}")


✅ Using embedding model: nebius/Qwen/Qwen3-Embedding-8B


In [8]:
## Local embedding model
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name = MY_CONFIG.EMBEDDING_MODEL
# )

## Step-5: Connect to Milvus

In [9]:
## Clear up any old data

from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)
    

/home/sujee/my-stuff/ai-alliance/data-prep-kit-examples/dpk-dev/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


✅ Connected to Milvus instance:  ./rag_1_dpk.db
✅ Cleared collection : docs


In [10]:
# connect llama-index to vector db

from llama_index.core import StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

✅ Connected Llama-index to Milvus instance:  ./rag_1_dpk.db


## Step-6: Save to DB

In [11]:
%%time 

# save chunks into vector db

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(
        nodes=nodes,
        storage_context=storage_context,
    )

print(f"Successfully stored {len(nodes)} chunks in Milvus collection '{MY_CONFIG.COLLECTION_NAME}'")


Successfully stored 91 chunks in Milvus collection 'docs'
CPU times: user 647 ms, sys: 90.6 ms, total: 737 ms
Wall time: 12.9 s


In [12]:
milvus_client.close()